# ALTEGRAD Challenge - Feature generation

*Abderrahim AIT-AZZI, Sébastien OHLEYER, Mickael SUTTON*

In [6]:
import pandas as pd
import numpy as np
import pickle
import gensim
from fuzzywuzzy import fuzz
from nltk.corpus import stopwords
from tqdm import tqdm
from scipy.stats import skew, kurtosis
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis
from nltk import word_tokenize
stop_words = stopwords.words('english')

## 1. WMD distance, Sent2vec, Glove

#### WMD Distance

In [17]:
def wmd(s1, s2):
    s1 = str(s1).lower().split()
    s2 = str(s2).lower().split()
    stop_words = stopwords.words('english')
    s1 = [w for w in s1 if w not in stop_words]
    s2 = [w for w in s2 if w not in stop_words]
    return model.wmdistance(s1, s2)

#### Sentence embedding

In [18]:
def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(model[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    return v / np.sqrt((v ** 2).sum())

### Import embedding model

#### Glove

In [ ]:
from gensim.models import KeyedVectors
# load the Stanford GloVe model
filename = './glove.42B.300d.txt.word2vec'
word_embedding_model_glove = KeyedVectors.load_word2vec_format(filename, binary=False)

#### W2V

In [5]:
model = gensim.models.KeyedVectors.load_word2vec_format('C:/Users/Amine/Desktop/MVA2017/ALTEGRAD/TP3/for moodle/code/GoogleNews-vectors-negative300.bin.gz', binary=True)

### Generate features for train data

We can generate some useful features:

    -Using the fuzz package to compute some ratio of string similarity between question 1 et question 2.
    -The length of questions
    -Word embedding of the questions and compute different distances: WMD, cosine, 

In [6]:
data_train = pd.read_csv('data/train.csv', sep=',',names = ["id", "qid1", "qid2", "question1","question2","is_duplicate"])
data_train = data_train.drop(['id', 'qid1', 'qid2'], axis=1)
data_train['len_q1'] = data_train.question1.apply(lambda x: len(str(x)))
data_train['len_q2'] = data_train.question2.apply(lambda x: len(str(x)))
data_train['diff_len'] = data_train.len_q1 - data_train.len_q2
data_train['len_char_q1'] = data_train.question1.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
data_train['len_char_q2'] = data_train.question2.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
data_train['len_word_q1'] = data_train.question1.apply(lambda x: len(str(x).split()))
data_train['len_word_q2'] = data_train.question2.apply(lambda x: len(str(x).split()))
data_train['common_words'] = data_train.apply(lambda x: len(set(str(x['question1']).lower().split()).intersection(set(str(x['question2']).lower().split()))), axis=1)
data_train['fuzz_qratio'] = data_train.apply(lambda x: fuzz.QRatio(str(x['question1']), str(x['question2'])), axis=1)
data_train['fuzz_WRatio'] = data_train.apply(lambda x: fuzz.WRatio(str(x['question1']), str(x['question2'])), axis=1)
data_train['fuzz_partial_ratio'] = data_train.apply(lambda x: fuzz.partial_ratio(str(x['question1']), str(x['question2'])), axis=1)
data_train['fuzz_partial_token_set_ratio'] = data_train.apply(lambda x: fuzz.partial_token_set_ratio(str(x['question1']), str(x['question2'])), axis=1)
data_train['fuzz_partial_token_sort_ratio'] = data_train.apply(lambda x: fuzz.partial_token_sort_ratio(str(x['question1']), str(x['question2'])), axis=1)
data_train['fuzz_token_set_ratio'] = data_train.apply(lambda x: fuzz.token_set_ratio(str(x['question1']), str(x['question2'])), axis=1)
data_train['fuzz_token_sort_ratio'] = data_train.apply(lambda x: fuzz.token_sort_ratio(str(x['question1']), str(x['question2'])), axis=1)

#### Embedding of the questions

In [8]:
model=word_embedding_model_glove
norm_model = word_embedding_model_glove
norm_model.init_sims(replace=True)

data_train['wmd'] = data_train.apply(lambda x: wmd(x['question1'], x['question2']), axis=1)

question1_vectors = np.zeros((data_train.shape[0], 300))
error_count = 0

for i, q in tqdm(enumerate(data_train.question1.values)):
    question1_vectors[i, :] = sent2vec(q)

question2_vectors  = np.zeros((data_train.shape[0], 300))
for i, q in tqdm(enumerate(data_train.question2.values)):
    question2_vectors[i, :] = sent2vec(q)

#### Compute distances

In [9]:
data_train['cosine_distance'] = [cosine(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

data_train['cityblock_distance'] = [cityblock(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

data_train['jaccard_distance'] = [jaccard(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

data_train['canberra_distance'] = [canberra(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

data_train['euclidean_distance'] = [euclidean(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

data_train['minkowski_distance'] = [minkowski(x, y, 3) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

data_train['braycurtis_distance'] = [braycurtis(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

data_train['skew_q1vec'] = [skew(x) for x in np.nan_to_num(question1_vectors)]
data_train['skew_q2vec'] = [skew(x) for x in np.nan_to_num(question2_vectors)]
data_train['kur_q1vec'] = [kurtosis(x) for x in np.nan_to_num(question1_vectors)]
data_train['kur_q2vec'] = [kurtosis(x) for x in np.nan_to_num(question2_vectors)]

#### Save features for train data

In [10]:
pickle.dump(question1_vectors, open('data/q1_glove.pkl', 'wb'), -1)
pickle.dump(question2_vectors, open('data/q2_glove.pkl', 'wb'), -1)

data_train.to_csv('data/train_features_glove.csv', index=False)

### Generete features for test data

In [11]:
data_test = pd.read_csv('data/test.csv', sep=',',names = ["id", "qid1", "qid2", "question1","question2"])

In [12]:
data_test['len_q1'] = data_test.question1.apply(lambda x: len(str(x)))
data_test['len_q2'] = data_test.question2.apply(lambda x: len(str(x)))
data_test['diff_len'] = data_test.len_q1 - data_test.len_q2
data_test['len_char_q1'] = data_test.question1.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
data_test['len_char_q2'] = data_test.question2.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
data_test['len_word_q1'] = data_test.question1.apply(lambda x: len(str(x).split()))
data_test['len_word_q2'] = data_test.question2.apply(lambda x: len(str(x).split()))
data_test['common_words'] = data_test.apply(lambda x: len(set(str(x['question1']).lower().split()).intersection(set(str(x['question2']).lower().split()))), axis=1)
data_test['fuzz_qratio'] = data_test.apply(lambda x: fuzz.QRatio(str(x['question1']), str(x['question2'])), axis=1)
data_test['fuzz_WRatio'] = data_test.apply(lambda x: fuzz.WRatio(str(x['question1']), str(x['question2'])), axis=1)
data_test['fuzz_partial_ratio'] = data_test.apply(lambda x: fuzz.partial_ratio(str(x['question1']), str(x['question2'])), axis=1)
data_test['fuzz_partial_token_set_ratio'] = data_test.apply(lambda x: fuzz.partial_token_set_ratio(str(x['question1']), str(x['question2'])), axis=1)
data_test['fuzz_partial_token_sort_ratio'] = data_test.apply(lambda x: fuzz.partial_token_sort_ratio(str(x['question1']), str(x['question2'])), axis=1)
data_test['fuzz_token_set_ratio'] = data_test.apply(lambda x: fuzz.token_set_ratio(str(x['question1']), str(x['question2'])), axis=1)
data_test['fuzz_token_sort_ratio'] = data_test.apply(lambda x: fuzz.token_sort_ratio(str(x['question1']), str(x['question2'])), axis=1)

In [13]:
norm_model = model
norm_model.init_sims(replace=True)
data_test['wmd'] = data_test.apply(lambda x: wmd(x['question1'], x['question2']), axis=1)

question1_vectors = np.zeros((data_test.shape[0], 300))
error_count = 0

for i, q in tqdm(enumerate(data_test.question1.values)):
    question1_vectors[i, :] = sent2vec(q)

question2_vectors  = np.zeros((data_test.shape[0], 300))
for i, q in tqdm(enumerate(data_test.question2.values)):
    question2_vectors[i, :] = sent2vec(q)

In [14]:
data_test['cosine_distance'] = [cosine(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

data_test['cityblock_distance'] = [cityblock(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

data_test['jaccard_distance'] = [jaccard(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

data_test['canberra_distance'] = [canberra(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

data_test['euclidean_distance'] = [euclidean(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

data_test['minkowski_distance'] = [minkowski(x, y, 3) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

data_test['braycurtis_distance'] = [braycurtis(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

data_test['skew_q1vec'] = [skew(x) for x in np.nan_to_num(question1_vectors)]
data_test['skew_q2vec'] = [skew(x) for x in np.nan_to_num(question2_vectors)]
data_test['kur_q1vec'] = [kurtosis(x) for x in np.nan_to_num(question1_vectors)]
data_test['kur_q2vec'] = [kurtosis(x) for x in np.nan_to_num(question2_vectors)]

#### Save features for test data

In [21]:
pickle.dump(question1_vectors, open('data/q1_glove_test.pkl', 'wb'), -1)
pickle.dump(question2_vectors, open('data/q2_glove_test.pkl', 'wb'), -1)
data_test.to_csv('data/test_features_glove.csv', index=False)

# 2. Page Rank

In [1]:
from features_engineering.pagerank import generate_pagerank
path = "./data"

In [2]:
generate_pagerank(path)

Apply to train...
Apply to test...
Main PR generator...
Apply to train...
Writing train...
Apply to test...
Writing test...
CSV written !


# 3. Question frequency

In [3]:
from features_engineering.question_freq import generate_question_freq
path = "./data"

In [4]:
generate_question_freq(path)

Writing train features...
Writing test features...
CSV written ! see:  ./data


# 4. Intersection of questions

In [5]:
from features_engineering.question_inter import generate_question_inter
path = "./data"

In [6]:
generate_question_inter(path)

Writing train features...
Writing test features...
CSV written ! see:  ./data



# 5. K cores

In [3]:
from features_engineering.kcores import generate_kcores

In [4]:
generate_kcores(path)

100%|██████████| 100279/100279 [00:01<00:00, 86552.54it/s]


Writing train features...
Writing test features...
CSV written ! see:  ./data  | suffix:  _kcores.csv


In [4]:
from features_engineering.question_kcores import generate_question_kcores
path='./data'

In [5]:
generate_question_kcores(path)

  0%|          | 0/29 [00:00<?, ?it/s]/Users/sebastienohleyer/Documents/ENS MVA/alte/features_engineering/question_kcores.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['kcores'][df.question.isin(ck)] = k
100%|██████████| 29/29 [00:21<00:00,  1.33it/s]


Writing train features...
Writing test features...
CSV written ! see:  ./data  | suffix:  _question_kcores.csv


# 6. TF-IDF

In [1]:
from features_engineering.tfidf import generate_tfidf
path = "./data/"

In [2]:
generate_tfidf(path)

Building Features
world_match
tfidf
tfidf_wm_stops
jaccard, wc_diff; wc_ratio, wc_diff_unique, wc_ratio_unique
wc_diff_unq_stop, wc_ratio_unique_stop
same_start, char_diff
char_diff_unq_stop
total_unique_words
total_unq_words_stop
char_ratio
world_match
tfidf
tfidf_wm_stops
jaccard, wc_diff; wc_ratio, wc_diff_unique, wc_ratio_unique
wc_diff_unq_stop, wc_ratio_unique_stop
same_start, char_diff
char_diff_unq_stop
total_unique_words
total_unq_words_stop
char_ratio
Writing train features...
Writing test features...
CSV written ! see:  ./data/  | suffix:  _tfidf.csv


# 7. Graph features

## 7.1 First set of features

In [3]:
#from features_engineering.graph_features import generate_graph_features
path = "./data"

import networkx as nx
import pandas as pd
import os
import numpy as np
from tqdm import tqdm
from networkx.exception import NetworkXNoPath

"""
Generate graph features for Quora question data. Features will be written in a csv file in path folder.

Args:
    path: folder containing train.csv and test.csv and to write csv features file.

Return:

"""
train = pd.read_csv(os.path.join(path,'train.csv'), sep=',',names = ["id", "qid1", "qid2", "question1","question2","is_duplicate"])
test =  pd.read_csv(os.path.join(path,'test.csv'), sep=',',names = ["id", "qid1", "qid2", "question1","question2"])

train = train.drop(['id','question1','question2', 'is_duplicate'], axis=1)
test = test.drop(['id','question1','question2'], axis=1)

train_test = pd.concat([train,test], ignore_index=True)

# Create graph
G=nx.Graph()

edge_list = []
for index, row in train_test.iterrows():
    edge_list.append([train_test['qid1'][index],train_test['qid2'][index]])

G.add_edges_from(edge_list)

print('Number of nodes:', G.number_of_nodes())
print('Number of edges:', G.number_of_edges())

Number of nodes: 58940
Number of edges: 100279


In [18]:
# Computing train features
print('Computing train features')
for index, row in tqdm(train.iterrows()):
    neigh_1 = G.neighbors(train['qid1'][index])
    neigh_2 = G.neighbors(train['qid2'][index])

    train.loc[index,'q1_neigh'] = len(neigh_1)
    train.loc[index,'q2_neigh'] = len(neigh_2)
    train.loc[index,'common_neigh'] = len(list(nx.common_neighbors(G,train['qid1'][index],train['qid2'][index])))
    train.loc[index,'distinct_neigh'] = len(neigh_1)+len(neigh_2)-len(list(nx.common_neighbors(G,train['qid1'][index],train['qid2'][index])))

    #train.loc[index,'all_simple_paths_3'] = len(list(nx.all_simple_paths(G,train['qid1'][index],train['qid2'][index])))

    train.loc[index,'clique_size'] = nx.node_clique_number(G,train['qid1'][index])
    #train.loc[index,'number_of_clique'] = nx.number_of_cliques(G,train['qid1'][index])

    #Cut the edge to compute features
    G.remove_edge(train['qid1'][index],train['qid2'][index])
    try:
        train.loc[index,'shortest_path'] = nx.shortest_path_length(G, train['qid1'][index], train['qid2'][index])
    except NetworkXNoPath:
        train.loc[index,'shortest_path'] = 10
    G.add_edge(train['qid1'][index],train['qid2'][index])

train = train.drop(['qid1','qid2'],axis=1)

print('Writing train features...')
train.to_csv(os.path.join(path,'train_graph_feat.csv'))

print('Computing test features')
for index, row in tqdm(test.iterrows()):
    neigh_1 = G.neighbors(test['qid1'][index])
    neigh_2 = G.neighbors(test['qid2'][index])

    test.loc[index,'q1_neigh'] = len(neigh_1)
    test.loc[index,'q2_neigh'] = len(neigh_2)
    test.loc[index,'common_neigh'] = len(list(nx.common_neighbors(G,test['qid1'][index],test['qid2'][index])))
    test.loc[index,'distinct_neigh'] = len(neigh_1)+len(neigh_2)-len(list(nx.common_neighbors(G,test['qid1'][index],test['qid2'][index])))

    #test.loc[index,'all_simple_paths_3'] = len(list(nx.all_simple_paths(G,test['qid1'][index],test['qid2'][index])))

    test.loc[index,'clique_size'] = nx.node_clique_number(G,test['qid1'][index])
    #test.loc[index,'number_of_clique'] = nx.number_of_cliques(G,test['qid1'][index])

    #Cut the edge to compute features
    G.remove_edge(test['qid1'][index],test['qid2'][index])
    try:
        test.loc[index,'shortest_path'] = nx.shortest_path_length(G, test['qid1'][index], test['qid2'][index])
    except NetworkXNoPath:
        test.loc[index,'shortest_path'] = 10
    G.add_edge(test['qid1'][index],test['qid2'][index])

test = test.drop(['qid1','qid2'],axis=1)

print('Writing test features...')    
test.to_csv(os.path.join(path,'test_graph_feat.csv'))

print('CSV written ! see: ', path, " | suffix: ", "_graph_feat.csv")

In [19]:
generate_graph_features(path)

9it [00:00, 89.32it/s]

Number of nodes: 58940
Number of edges: 100279
Computing train features


80100it [10:25, 128.15it/s]


Writing train features...


15it [00:00, 145.07it/s]

Computing test features


20179it [02:06, 159.67it/s]


Writing test features...
CSV written ! see:  ./data  | suffix:  _graph_feat.csv


## Second set of features

In [ ]:
#from features_engineering.graph_features2 import generate_graph_features2

# 8. N-grams

In [1]:
from features_engineering.cooccurence_distinct_ngram import generate_cooccurence_distinct_ngram

In [2]:
path = './data'
generate_cooccurence_distinct_ngram(path,3)

8it [00:00, 75.03it/s]

Applying to train...


80100it [18:10, 73.44it/s]


Writing train features...


11it [00:00, 103.50it/s]

Applying to test...


20179it [02:46, 121.05it/s]


Writing test features...
CSV written ! see:  ./data  | suffix:  _3gram_feat.csv


# 9. Word features

See with Abderrahim

In [1]:
path = ''

from collections import defaultdict
import numpy as np
import pandas as pd
import xgboost as xgb
from nltk.corpus import stopwords
from collections import Counter
from sklearn.metrics import log_loss

def add_word_count(x, df, word):
	x['q1_' + word] = df['question1'].apply(lambda x: (word in str(x).lower())*1)
	x['q2_' + word] = df['question2'].apply(lambda x: (word in str(x).lower())*1)
	x[word + '_both'] = x['q1_' + word] * x['q2_' + word]
    
def get_weight(count, eps=10000, min_count=2):
    return 0 if count < min_count else 1 / (count + eps)

def word_shares(row):

	q1_list = str(row['question1']).lower().split()
	q1 = set(q1_list)
	q1words = q1.difference(stops)
	if len(q1words) == 0:
		return '0:0:0:0:0:0:0:0'
        
	q2_list = str(row['question2']).lower().split()
	q2 = set(q2_list)
	q2words = q2.difference(stops)
	if len(q2words) == 0:
		return '0:0:0:0:0:0:0:0'

	words_hamming = sum(1 for i in zip(q1_list, q2_list) if i[0]==i[1])/max(len(q1_list), len(q2_list))
    
	q1stops = q1.intersection(stops)
	q2stops = q2.intersection(stops)
	q1_2gram = set([i for i in zip(q1_list, q1_list[1:])])
	q2_2gram = set([i for i in zip(q2_list, q2_list[1:])])
	shared_2gram = q1_2gram.intersection(q2_2gram)
	shared_words = q1words.intersection(q2words)
	shared_weights = [weights.get(w, 0) for w in shared_words]
	q1_weights = [weights.get(w, 0) for w in q1words]
	q2_weights = [weights.get(w, 0) for w in q2words]
	total_weights = q1_weights + q1_weights
	
	R1 = np.sum(shared_weights) / np.sum(total_weights) #tfidf share
	R2 = len(shared_words) / (len(q1words) + len(q2words) - len(shared_words)) #count share
	R31 = len(q1stops) / len(q1words) #stops in q1
	R32 = len(q2stops) / len(q2words) #stops in q2
	Rcosine_denominator = (np.sqrt(np.dot(q1_weights,q1_weights))*np.sqrt(np.dot(q2_weights,q2_weights)))
	Rcosine = np.dot(shared_weights, shared_weights)/Rcosine_denominator
	if len(q1_2gram) + len(q2_2gram) == 0:
		R2gram = 0
	else:
		R2gram = len(shared_2gram) / (len(q1_2gram) + len(q2_2gram))
	return '{}:{}:{}:{}:{}:{}:{}:{}'.format(R1, R2, len(shared_words), R31, R32, R2gram, Rcosine, words_hamming)

df_train =pd.read_csv('train.csv', sep=',',names = ["id", "qid1", "qid2", "question1","question2","is_duplicate"])
df_train = df_train.fillna(' ')

df_test=  pd.read_csv('test.csv', sep=',',names = ["id", "qid1", "qid2", "question1","question2"])
df_test = df_test.fillna(' ')

# explore
stops = set(stopwords.words("english"))

df_train['question1'] = df_train['question1'].map(lambda x: str(x).lower().split())
df_train['question2'] = df_train['question2'].map(lambda x: str(x).lower().split())

train_qs = pd.Series(df_train['question1'].tolist() + df_train['question2'].tolist())

words = [x for y in train_qs for x in y]
counts = Counter(words)
weights = {word: get_weight(count) for word, count in counts.items()}

df = pd.concat([df_train, df_test])
df['word_shares'] = df.apply(word_shares, axis=1, raw=True)
x = pd.DataFrame()

x['word_match']       = df['word_shares'].apply(lambda x: float(x.split(':')[0]))
x['word_match_2root'] = np.sqrt(x['word_match'])
x['tfidf_word_match'] = df['word_shares'].apply(lambda x: float(x.split(':')[1]))
x['shared_count']     = df['word_shares'].apply(lambda x: float(x.split(':')[2]))

x['stops1_ratio']     = df['word_shares'].apply(lambda x: float(x.split(':')[3]))
x['stops2_ratio']     = df['word_shares'].apply(lambda x: float(x.split(':')[4]))
x['shared_2gram']     = df['word_shares'].apply(lambda x: float(x.split(':')[5]))
x['cosine']           = df['word_shares'].apply(lambda x: float(x.split(':')[6]))
x['words_hamming']    = df['word_shares'].apply(lambda x: float(x.split(':')[7]))
x['diff_stops_r']     = x['stops1_ratio'] - x['stops2_ratio']

x['len_q1'] = df['question1'].apply(lambda x: len(str(x)))
x['len_q2'] = df['question2'].apply(lambda x: len(str(x)))
x['diff_len'] = x['len_q1'] - x['len_q2']
	
x['caps_count_q1'] = df['question1'].apply(lambda x:sum(1 for i in str(x) if i.isupper()))
x['caps_count_q2'] = df['question2'].apply(lambda x:sum(1 for i in str(x) if i.isupper()))
x['diff_caps'] = x['caps_count_q1'] - x['caps_count_q2']

x['len_char_q1'] = df['question1'].apply(lambda x: len(str(x).replace(' ', '')))
x['len_char_q2'] = df['question2'].apply(lambda x: len(str(x).replace(' ', '')))
x['diff_len_char'] = x['len_char_q1'] - x['len_char_q2']

x['len_word_q1'] = df['question1'].apply(lambda x: len(str(x).split()))
x['len_word_q2'] = df['question2'].apply(lambda x: len(str(x).split()))
x['diff_len_word'] = x['len_word_q1'] - x['len_word_q2']

x['avg_world_len1'] = x['len_char_q1'] / x['len_word_q1']
x['avg_world_len2'] = x['len_char_q2'] / x['len_word_q2']
x['diff_avg_word'] = x['avg_world_len1'] - x['avg_world_len2']

x['exactly_same'] = (df['question1'] == df['question2']).astype(int)
# x['duplicated'] = df.duplicated(['question1','question2']).astype(int)
add_word_count(x, df,'how')
add_word_count(x, df,'what')
add_word_count(x, df,'which')
add_word_count(x, df,'who')
add_word_count(x, df,'where')
add_word_count(x, df,'when')
add_word_count(x, df,'why')

print(x.columns)
print(x.describe())

x_train_question = x[:df_train.shape[0]]
x_test_question  = x[df_train.shape[0]:]

/Users/sebastienohleyer/anaconda3/envs/nlp/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


FileNotFoundError: File b'train.csv' does not exist

# 10. SpaCy

In [23]:
from features_engineering.spacy_features import generate_spacy_features

In [25]:
generate_spacy_features(path)

4it [00:00, 32.93it/s]

Applying to train...


80100it [48:53, 27.31it/s]


Writing train features...


2it [00:00, 18.54it/s]

Applying to train...


20179it [11:35, 29.02it/s]


Writing test features...
CSV written ! see:  ./data  | suffix:  _spacy_features.csv
